In [48]:
"""
Live power breakdown

    "powerProduction" (in MW) represents the electricity produced in the zone, broken down by production type
    "powerConsumption" (in MW) represents the electricity consumed in the zone, after taking into account imports and exports, and broken down by production type.
    "powerExport" and "Power import" (in MW) represent the physical electricity flows at the zone border
    "renewablePercentage" and "fossilFreePercentage" refers to the % of the power consumption breakdown coming from renewables or fossil-free power plants (renewables and nuclear) It can either be queried by zone identifier or by geolocation.
"""
import json
import os
import pandas as pd
import requests
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import numpy as np
pd.set_option('display.max_columns', 50)
load_dotenv()

api_key = os.getenv('ELECTRICITYMAPS_API')
endpoint = 'https://api.electricitymap.org/v3/power-breakdown/latest?'




header = {'auth-token': api_key}



In [40]:
query = 'zone=US-NW-TPWR'
url = endpoint + query
breakdown_df = pd.DataFrame()
response = requests.get(url, header).json()


In [41]:
response

{'zone': 'US-NW-TPWR',
 'datetime': '2024-05-18T18:00:00.000Z',
 'updatedAt': '2024-05-18T17:51:58.173Z',
 'createdAt': '2024-05-15T18:51:54.838Z',
 'powerConsumptionBreakdown': {'nuclear': 0,
  'geothermal': 0,
  'biomass': 0,
  'coal': 0,
  'wind': 0,
  'solar': 0,
  'hydro': 273,
  'gas': 0,
  'oil': 0,
  'unknown': 0,
  'hydro discharge': 0,
  'battery discharge': 0},
 'powerProductionBreakdown': {'nuclear': None,
  'geothermal': None,
  'biomass': None,
  'coal': None,
  'wind': None,
  'solar': None,
  'hydro': 273,
  'gas': None,
  'oil': None,
  'unknown': None,
  'hydro discharge': None,
  'battery discharge': None},
 'powerImportBreakdown': {},
 'powerExportBreakdown': {},
 'fossilFreePercentage': 100,
 'renewablePercentage': 100,
 'powerConsumptionTotal': 273,
 'powerProductionTotal': 273,
 'powerImportTotal': None,
 'powerExportTotal': None,
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

In [13]:
consumption_breakdown_df = pd.DataFrame(response['powerConsumptionBreakdown'], index=[0])
consumption_breakdown_df['zone'] = response['zone']
consumption_breakdown_df['date'] = response['datetime']

In [14]:
consumption_breakdown_df.head()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro discharge,battery discharge,zone,date
0,1304,0,93,256,29,1073,273,389,0,53,94,0,HU,2024-05-18T17:00:00.000Z


In [15]:
production_breakdown_df = pd.DataFrame(response['powerProductionBreakdown'], index=[0])
production_breakdown_df['zone'] = response['zone']
production_breakdown_df['date'] = response['datetime']

In [16]:
production_breakdown_df.head()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro discharge,battery discharge,zone,date
0,1426,0,84,280,4,1104,32,424,0,57,None,None,HU,2024-05-18T17:00:00.000Z


In [18]:
berakdown_general = {
    'zone': response['zone'],
    'datetime': response['datetime'],
    'fossilFreePercentage': response['fossilFreePercentage'],
    'renewablePercentage': response['renewablePercentage'],
    'powerConsumptionTotal': response['powerConsumptionTotal'],
    'powerProductionTotal': response['powerConsumptionTotal'],
    'powerImportTotal': response['powerImportTotal'],
    'powerExportTotal': response['powerExportTotal'],
    'isEstimated': response['isEstimated'],
    'estimationMethod': response['estimationMethod']
    }

In [20]:
breakdown_general_df = pd.DataFrame(berakdown_general, index=[0])

In [21]:
breakdown_general_df.head()

,zone,datetime,fossilFreePercentage,renewablePercentage,powerConsumptionTotal,powerProductionTotal,powerImportTotal,powerExportTotal,isEstimated,estimationMethod
0,HU,2024-05-18T17:00:00.000Z,80,44,3564,3564,1129,976,True,TIME_SLICER_AVERAGE


In [38]:
zone_endpoint = 'https://api.electricitymap.org/v3/zones'
response = requests.get(zone_endpoint, header)
zone_data = response.json()

consumption_breakdown_df = pd.DataFrame()

for key, value in zone_data.items():
    query = 'zone=' + key
    url = endpoint + query
    response = requests.get(url, header).json()
    
    if len(response) != 1:
        temp_df = pd.DataFrame(response['powerConsumptionBreakdown'], index=[0])
        temp_df['zone'] = response['zone']
        temp_df['date'] = response['datetime']
        consumption_breakdown_df = pd.concat([consumption_breakdown_df, temp_df], ignore_index=True)
        
        temp_df = pd.DataFrame(response['powerProductionBreakdown'], index=[0])
        temp_df['zone'] = response['zone']
        temp_df['date'] = response['datetime']
        production_breakdown_df = pd.concat([production_breakdown_df, temp_df], ignore_index=True)

        berakdown_general = {
            'zone': response['zone'],
            'datetime': response['datetime'],
            'fossilFreePercentage': response['fossilFreePercentage'],
            'renewablePercentage': response['renewablePercentage'],
            'powerConsumptionTotal': response['powerConsumptionTotal'],
            'powerProductionTotal': response['powerConsumptionTotal'],
            'powerImportTotal': response['powerImportTotal'],
            'powerExportTotal': response['powerExportTotal'],
            'isEstimated': response['isEstimated'],
            'estimationMethod': response['estimationMethod']
        }
        temp_df = pd.DataFrame(berakdown_general, index=[0])
        breakdown_general_df = pd.concat([breakdown_general_df, temp_df], ignore_index=True)
       

In [34]:
consumption_breakdown_df.sample(5)


,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro discharge,battery discharge,zone,date
59,0,0,2,0,0,0,0,0,22,0,0,0,ES-ML,2024-05-18T18:00:00.000Z
89,0,626,967,422,362,2066,8854,8756,50,1410,3458,0,IT,2024-05-18T18:00:00.000Z
213,8193,0,0,3719,0,2423,547,14001,0,124,0,0,US-SE-SOCO,2024-05-18T18:00:00.000Z
162,0,854,664,10753,1284,0,13814,5110,96,74,0,0,TR,2024-05-18T18:00:00.000Z
149,21164,0,0,230,1,1,13368,61,2,53830,0,0,RU-1,2024-05-18T18:00:00.000Z


In [43]:
production_breakdown_df.head(5)

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro discharge,battery discharge,zone,date
0,1426,0,84,280,4,1104,32,424,0,57,None,None,HU,2024-05-18T17:00:00.000Z
1,2307,0,None,0,0,0,0,21090,0,51,None,None,AE,2024-05-18T18:00:00.000Z
2,1709,None,98,None,376,846,5119,None,None,8773,None,None,AR,2024-05-18T18:00:00.000Z
3,None,0,324,0,588,1024,4801,16,0,22,1513,None,AT,2024-05-18T18:00:00.000Z
4,None,None,48,15158,5072,0,329,1354,0,1,-103,-32,AU,2024-05-18T18:00:00.000Z


In [50]:
production_breakdown_df.fillna(0)

C:\Users\chris\AppData\Local\Temp\ipykernel_26304\3270342679.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  production_breakdown_df.fillna(0)


,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro discharge,battery discharge,zone,date
0,1426,0,84,280,4,1104,32,424,0,57,0,0,HU,2024-05-18T17:00:00.000Z
1,2307,0,0,0,0,0,0,21090,0,51,0,0,AE,2024-05-18T18:00:00.000Z
2,1709,0,98,0,376,846,5119,0,0,8773,0,0,AR,2024-05-18T18:00:00.000Z
3,0,0,324,0,588,1024,4801,16,0,22,1513,0,AT,2024-05-18T18:00:00.000Z
4,0,0,48,15158,5072,0,329,1354,0,1,-103,-32,AU,2024-05-18T18:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,8162,0,0,1782,0,604,4154,4672,0,10,0,0,US-TEN-TVA,2024-05-18T18:00:00.000Z
222,3604,0,0,6405,3169,10515,134,28218,0,0,0,0,US-TEX-ERCO,2024-05-18T18:00:00.000Z
223,0,0,215,0,466,176,1004,0,0,0,0,0,UY,2024-05-18T18:00:00.000Z
224,0,0,0,586,0,0,0,0,0,0,0,0,XK,2024-05-18T18:00:00.000Z


In [39]:
breakdown_general_df.head()

,zone,datetime,fossilFreePercentage,renewablePercentage,powerConsumptionTotal,powerProductionTotal,powerImportTotal,powerExportTotal,isEstimated,estimationMethod
0,ZA,2024-05-18T18:00:00.000Z,15,11,23924,23924,0,0,True,TIME_SLICER_AVERAGE
1,AE,2024-05-18T18:00:00.000Z,10,0,22985,22985,0,0,True,RECONSTRUCT_PRODUCTION_FROM_CONSUMPTION
2,AR,2024-05-18T18:00:00.000Z,47,37,16657,16657,74,0,False,None
3,AT,2024-05-18T18:00:00.000Z,100,100,6449,6449,0,1760,True,TIME_SLICER_AVERAGE
4,AU,2024-05-18T18:00:00.000Z,25,25,21324,21324,0,0,True,TIME_SLICER_AVERAGE
